In [275]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification,DefaultDataCollator
from datasets import load_dataset
import re
import tensorflow as tf
from tensorflow import keras
import datetime

In [ ]:
##download distilbert from the git rep
#### git lfs install
#### git clone https://huggingface.co/bert-base-uncased

In [ ]:
checkpoint="E:\\python_projects\\transformers\\distilbert-base-uncased"
input_file_sentiment="E:\\python_projects\\Springboard\datasets\\twitter_sentiment140\\training_140_sentiment.csv"

In [ ]:
tokenizer=AutoTokenizer.from_pretrained(checkpoint)
model=TFAutoModelForSequenceClassification.from_pretrained(checkpoint)

In [ ]:
sentiment_colls=['sentiment','tweet_id','tweet_date','query','username','tweet']
tweets_ds=load_dataset("csv",data_files=input_file_sentiment,encoding='cp1252',column_names=sentiment_colls,usecols=['sentiment','tweet'])
tweets_ds

In [ ]:
set(tweets_ds['train']['sentiment'])

In [ ]:
tweets_ds=tweets_ds.map(lambda x:{'labels':[1 if w==4 else w for w in x['sentiment']]},batched=True)


In [ ]:
set(tweets_ds['train']['labels'])

In [ ]:
tweets_ds_dict=tweets_ds['train'].train_test_split(test_size=0.3)

In [ ]:
tweets_ds_dict

In [255]:
#here we are not padding the inputs as this is not efficient. This also means we cannot use return_tensors='tf'
def tokenize_function(text):
    return tokenizer(text["tweet"], truncation=True,padding="max_length",max_length=512)

In [256]:
tokenized_datasets = tweets_ds_dict.map(tokenize_function, batched=True)

  0%|          | 0/1120 [00:00<?, ?ba/s]

  0%|          | 0/480 [00:00<?, ?ba/s]

In [257]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['sentiment', 'tweet', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 1120000
    })
    test: Dataset({
        features: ['sentiment', 'tweet', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 480000
    })
})

In [258]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_test_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

In [ ]:
for rec in small_train_dataset:
    print(len(rec['input_ids']),len(rec['attention_mask']))

In [260]:
data_collator = DefaultDataCollator(return_tensors='tf')

In [261]:
tokenized_datasets=tokenized_datasets.remove_columns(['sentiment','tweet'])


In [262]:
tokenized_datasets['train'].column_names

['labels', 'input_ids', 'attention_mask']

In [263]:
tf_train_dataset = small_train_dataset.to_tf_dataset(
    columns=["attention_mask", "input_ids"],
    label_cols=["labels"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)

In [265]:
tf_validation_dataset = small_test_dataset.to_tf_dataset(
    columns=["attention_mask", "input_ids"],
    label_cols=["labels"],
    shuffle=False,
    collate_fn=data_collator,
    batch_size=8,
)

In [273]:
   
    model.compile(loss="binary_crossentropy",
                   optimizer=keras.optimizers.SGD(learning_rate=5e-5),
                   metrics=["accuracy"])

In [277]:
model.summary()

Model: "tf_distil_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 distilbert (TFDistilBertMai  multiple                 66362880  
 nLayer)                                                         
                                                                 
 pre_classifier (Dense)      multiple                  590592    
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
 dropout_19 (Dropout)        multiple                  0         
                                                                 
Total params: 66,955,010
Trainable params: 66,955,010
Non-trainable params: 0
_________________________________________________________________


In [276]:
start_time=datetime.datetime.now()
print(f'start time:{start_time}')
history=model.fit(tf_train_dataset, validation_data=tf_validation_dataset, epochs=3)
print(f'Time taken:{datetime.datetime.now()-start_time}')

start time:2022-08-23 00:22:30.504445
Epoch 1/3
125/125 [==============================] - 3938s 31s/step - loss: 4.1648 - accuracy: 0.4990 - val_loss: 4.4408 - val_accuracy: 0.4630
Epoch 2/3
125/125 [==============================] - 3930s 32s/step - loss: 4.3322 - accuracy: 0.4790 - val_loss: 4.4410 - val_accuracy: 0.4630
Epoch 3/3
125/125 [==============================] - 3931s 32s/step - loss: 4.3338 - accuracy: 0.4790 - val_loss: 4.4417 - val_accuracy: 0.4630
Time taken:3:16:38.543417
